In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.2.0
Eager execution: True


In [3]:
import numpy as np
import sklearn
from sklearn import preprocessing

In [4]:
model = "gat"
hidden_units = "16,16"
heads = "8,8,1"
epochs = 5
lr = 0.1
dropout = 0.2
file_dir = '/Users/lsj/Downloads/DeepInf-homework/weibo'
batch = 1024
train_ratio = 75
vaild_ratio = 12.5
instance_normalization = True
use_vertex_feature = True


In [5]:
def load_w2v_feature(file, max_idx=0):
    with open(file, "rb") as f:
        nu = 0
        for line in f:
            content = line.strip().split()
            nu += 1
            if nu == 1:
                n, d = int(content[0]), int(content[1])
                feature = [[0.] * d for i in range(max(n, max_idx + 1))]
                continue
            index = int(content[0])
            while len(feature) <= index:
                feature.append([0.] * d)
            for i, x in enumerate(content[1:]):
                feature[index][i] = float(x)
    for item in feature:
        assert len(item) == d
    return np.array(feature, dtype=np.float32)

In [6]:
graphs = np.load(os.path.join(file_dir, "adjacency_matrix.npy")).astype(np.float32)
# self-loop trick, the input graphs should have no self-loop
identity = np.identity(graphs.shape[1])
graphs += identity
graphs[graphs != 0] = 1.0
if model == "gat":
   graphs = graphs.astype(np.dtype('B'))
elif model == "gcn":
    # normalized graph laplacian for GCN: D^{-1/2}AD^{-1/2}
    for i in range(len(graphs)):
        graph = graphs[i]
        d_root_inv = 1. / np.sqrt(np.sum(graph, axis=1))
        graph = (graph.T * d_root_inv).T * d_root_inv
        graphs[i] = graph
else:
    raise NotImplementedError

# wheather a user has been influenced
# wheather he/she is the ego user
influence_features = np.load(
                os.path.join(file_dir, "influence_feature.npy")).astype(np.float32)

labels = np.load(os.path.join(file_dir, "label.npy"))

vertices = np.load(os.path.join(file_dir, "vertex_id.npy"))

graphs, influence_features, labels,vertices = sklearn.utils.shuffle(
                        graphs, influence_features,
                        labels, vertices,
                        random_state=47)

vertex_features = np.load(os.path.join(file_dir, "vertex_feature.npy"))
vertex_features = preprocessing.scale(vertex_features)

embedding_path = os.path.join(file_dir, "deepwalk.emb_64")
max_vertex_idx = np.max(vertices)
embedding = load_w2v_feature(embedding_path, max_vertex_idx)



In [7]:
print(graphs.shape)
print(labels.shape)
print(influence_features.shape)
print(vertex_features.shape)
print(embedding.shape)
print(labels[:2])


(779164, 50, 50)
(779164,)
(779164, 50, 2)
(1784073, 7)
(1784024, 64)
[1 1]


In [8]:

class GAT(keras.layers.Layer):
    def __init__(self, n_head, f_in, f_out, attn_dropout):
        super(GAT, self).__init__()
        self.n_head = n_head
        w_init = tf.keras.initializers.GlorotUniform()
        self.w = tf.Variable(
            initial_value=w_init(shape=(n_head, f_in, f_out), dtype="float32"),
            trainable=True,
        )
        self.a_src = tf.Variable(
            initial_value=w_init(shape=(n_head, f_out, 1), dtype="float32"),
            trainable=True,
        )
        self.a_dst = tf.Variable(
            initial_value=w_init(shape=(n_head, f_out, 1), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(f_out,), dtype="float32"), trainable=True
        )
        self.dropout = attn_dropout
    def call(self, h, adj):
   #     h, adj = inputs
        bs, n = h.shape[:2]
        h_prime = tf.matmul(tf.expand_dims(h,1),self.w)
        attn_src = tf.matmul(tf.math.tanh(h_prime),self.a_src)
        attn_dst = tf.matmul(tf.math.tanh(h_prime),self.a_dst)
        s1 = attn_src.shape
        s2 = attn_dst.shape
        attn_src_b = tf.broadcast_to(attn_src,[s1[0],s1[1],s1[2],n])
        attn_dst_b = tf.broadcast_to(attn_dst,[s2[0],s2[1],s2[2],n])
        attn_dst_permute = tf.keras.layers.Permute((1,3,2))(attn_dst_b)
        attn = attn_src_b + attn_dst_permute

        attn = tf.nn.leaky_relu(attn,alpha=0.2)
        ## mask = 1 - tf.expand_dims(adj,1)
        ## todo: mask fill
        
        attn  = tf.nn.softmax(attn,axis=-1)
        attn = tf.nn.dropout(attn,self.dropout)
        return tf.matmul(attn, h_prime)


In [16]:
vertices_inputs = keras.Input(shape=(None,),name='vertices')
#print(vertices_inputs.op)
feature_inputs = keras.Input(shape=(50,2),batch_size=32,name='feature')
print(feature_inputs)
graph_inputs = keras.Input(shape=(50,50),name='graph')
print(graph_inputs)
##todo: emb init
#emb = keras.layers.Embedding(embedding.shape[0],embedding.shape[1],embeddings_initializer=xxxx)
##todo: instance normalization
##emb = tfa.layers.InstanceNormalization(emb)
#x = keras.layers.Concatenate(feature_inputs,emb)
#x = feature_inputs
#x = keras.layers.Embedding(vertex_features.shape(0),ertex_features.shape(1), embeddings_initializer=xxx)

##todo: multi head
#x = keras.layers.AdditiveAttention(x,graph_inputs) 
        # n_units [2,16,16,2] after add emedding shuld be 2 + 64 + 7 = 73
        # n head [8,8,1]
        # 8,2,16,0.2
        # 8,16*8,16, 0.2
        # 1,16*8,2, 0.2

x = GAT(8,2,16,0.2)(feature_inputs,graph_inputs) # (32, 8, 50, 16)
x = tf.transpose(x,perm=[0,2,1,3])  # (32, 50, 8, 16)
x = tf.reshape(x,[32,50,-1])   # (32, 50, 128)
# todo: elu(x) + dropout

x = GAT(8,16*8,16,0.2)(x,graph_inputs) # (32, 8, 50, 16)
x = tf.transpose(x,perm=[0,2,1,3]) # (32, 50, 8, 16)
x = tf.reshape(x,[32,50,-1])   # (32, 50, 128)
# todo: elu(x) + dropout

x = GAT(1,16*8,2,0.2)(x,graph_inputs) ((32, 1, 50, 2))
x = tf.reduce_mean(x,axis=1)  # (32, 50, 2)
output = tf.nn.softmax(x,name='output')


Tensor("feature_1:0", shape=(32, 50, 2), dtype=float32)
Tensor("graph_1:0", shape=(None, 50, 50), dtype=float32)
(32, 8, 50, 16)
(32, 50, 8, 16)
(32, 50, 128)
(32, 8, 50, 16)
(32, 50, 8, 16)
(32, 50, 128)
(32, 1, 50, 2)
(32, 50, 2)


In [39]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
       
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            y_pred = y_pred[:,-1,:]
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            print(loss)
        # Compute gradients
        trainable_vars = self.trainable_variables
        ### realy work?
        trainable_vars = [ x for x in filter(lambda i: 'embedding' not in i.name ,trainable_vars)]
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [40]:
model = CustomModel(
    inputs=[vertices_inputs, feature_inputs, graph_inputs],
    outputs=output,
)

In [41]:
## pytorch nllloss maybe == tf SparseCategoricalCrossentropy
model.compile(
    optimizer=keras.optimizers.Adagrad(lr=lr),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)


In [23]:
model.summary()

Model: "custom_model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
feature (InputLayer)            [(32, 50, 2)]        0                                            
__________________________________________________________________________________________________
graph (InputLayer)              [(None, 50, 50)]     0                                            
__________________________________________________________________________________________________
gat_3 (GAT)                     (32, 8, 50, 16)      528         feature[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Transpose_2 (Tensor [(32, 50, 8, 16)]    0           gat_3[0][0]                      
_____________________________________________________________________________________

In [42]:
model.fit(
    {"vertices": vertices , "feature":influence_features , "graph": graphs},
    {"tf_op_layer_output": labels},
    epochs=1,
    batch_size=32,
)

({'vertices': <tf.Tensor 'IteratorGetNext:2' shape=(None, 50) dtype=int64>, 'feature': <tf.Tensor 'IteratorGetNext:0' shape=(None, 50, 2) dtype=float32>, 'graph': <tf.Tensor 'IteratorGetNext:1' shape=(None, 50, 50) dtype=uint8>}, {'tf_op_layer_output_1': <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>})
y_pred shape
(32, 50, 2)
(32, 2)
data
{'tf_op_layer_output_1': <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>}
Tensor("strided_slice:0", shape=(32, 2), dtype=float32)
Tensor("sparse_categorical_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
Variable:0
[<tf.Variable 'Variable:0' shape=(8, 2, 16) dtype=float32>, <tf.Variable 'Variable:0' shape=(8, 16, 1) dtype=float32>, <tf.Variable 'Variable:0' shape=(8,

AttributeError: in user code:

    /Users/lsj/Library/Python/3.7/lib/python/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    <ipython-input-39-d84ba0cfec1f>:32 train_step  *
        print(k.values)

    AttributeError: 'Tensor' object has no attribute 'values'
